In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import pandas_datareader as pdr
import pytictoc as tt
from tickers500 import tickers500
from tickerTA import Ticker
from tickerTA import TechnicalAnalysis
from tradingStrategy1 import TradingStrategy1

# This is to avoid the SettingWithCopyWarning in pandas
pd.options.mode.copy_on_write = True 


In [2]:
# This class is used to parse the daily data from the Yahoo Finance API
# to implement the Ticker class, TechnicalAnalysis class and TradingStrategy1 class
# After processing all the data, it will output the current Buy/Sell recommendations list
# for the 500 S&P companies
# Also it will include the price change since the last recommendation

class Recommendations:
    def __init__(self, start = pd.to_datetime('today') - pd.DateOffset(months=12), end = pd.to_datetime('today')):
        self.start = start
        self.end = end
        self.tickers500 = tickers500
        self.buy_recommendations_df = pd.DataFrame()
        self.sell_recommendations_df = pd.DataFrame()
        self.get_stock_data()
        self.current_recommendations()

    def get_stock_data(self):
        self.tradingstrategy1_df = pd.DataFrame()
        self.signals_df = pd.DataFrame()
        for ticker in self.tickers500:
            stock = Ticker(ticker, self.start, self.end)
            # stock_df = stock_df._append(stock.df)
            techA = TechnicalAnalysis(stock)
            tradingS = TradingStrategy1(techA)
            self.signals_df = self.signals_df._append(tradingS.all_signals_df)
            self.tradingstrategy1_df = self.tradingstrategy1_df._append(tradingS.trades_df)
        self.signals_df = self.signals_df.sort_values(by='Date')

    def current_recommendations(self):
        for _, row in self.signals_df.iterrows():
            if row['Buy/Sell'] == 'Buy':
                self.buy_recommendations_df = self.buy_recommendations_df._append(row)
            elif row['Buy/Sell'] == 'Sell':
                self.sell_recommendations_df = self.sell_recommendations_df._append(row)
        self.buy_recommendations_df = self.buy_recommendations_df.sort_values(by='Date', ascending=False)
        self.sell_recommendations_df = self.sell_recommendations_df.sort_values(by='Date', ascending=False)
        # self.buy_recommendations_df = self.buy_recommendations_df.drop_duplicates(subset=['Ticker'])
        # self.sell_recommendations_df = self.sell_recommendations_df.drop_duplicates(subset=['Ticker'])

In [3]:
rec = Recommendations()


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FOX?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=FOX&crumb=s8OEDvcJwS9
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/NWS?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=NWS&crumb=s8OEDvcJwS9
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SOLV?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=SOLV&crumb=s8OEDvcJwS9
c:\Users\chris\Documents\Unreal Projects\AlgorithmicTrading\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1057: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\chris\Documents\Unreal Projects\AlgorithmicTrading\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1081: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) ==

In [4]:

print(rec.signals_df)

   Ticker Buy/Sell                 Date   Price
0       C      Buy  2023-05-26 00:00:00   44.46
0     BMY      Buy  2023-05-26 00:00:00   64.51
0     EMR      Buy  2023-05-26 00:00:00   79.28
0     MRK      Buy  2023-05-26 00:00:00   112.0
0      EW      Buy  2023-05-26 00:00:00   81.54
..    ...      ...                  ...     ...
3     WBA     Sell  2024-04-01 00:00:00   21.57
5     NEM     Sell  2024-04-01 00:00:00   36.96
5     AWK     Sell  2024-04-01 00:00:00  122.44
5     EIX     Sell  2024-04-01 00:00:00   70.58
7     LNT     Sell  2024-04-01 00:00:00   50.61

[2045 rows x 4 columns]


In [5]:
rec.buy_recommendations_df.to_csv('CSV/buy_recommendations.csv')
rec.buy_recommendations_df


,Ticker,Buy/Sell,Date,Price
4,CSX,Buy,2024-03-27,36.34
4,MDLZ,Buy,2024-03-27,70.05
4,UPS,Buy,2024-03-27,144.39
6,ES,Buy,2024-03-27,56.91
6,CNP,Buy,2024-03-27,27.45
...,...,...,...,...
0,GIS,Buy,2023-05-26,84.36
0,CME,Buy,2023-05-26,176.70
0,WBD,Buy,2023-05-26,11.17
0,CINF,Buy,2023-05-26,98.26


In [6]:
rec.sell_recommendations_df.to_csv('CSV/sell_recommendations.csv')
rec.sell_recommendations_df


,Ticker,Buy/Sell,Date,Price
7,LNT,Sell,2024-04-01,50.61
5,EIX,Sell,2024-04-01,70.58
5,AWK,Sell,2024-04-01,122.44
5,NEM,Sell,2024-04-01,36.96
3,WBA,Sell,2024-04-01,21.57
...,...,...,...,...
1,PCAR,Sell,2023-06-07,74.83
1,RHI,Sell,2023-06-07,71.89
1,ELV,Sell,2023-06-06,480.38
1,CAT,Sell,2023-06-05,226.99


In [7]:
rec.tradingstrategy1_df.to_csv('CSV/tradingstrategy1.csv')
rec.tradingstrategy1_df

,Ticker,Buy Date,Buy Price,Buy P/E Ratio,Buy Fwd P/E Ratio,Buy P/B Ratio,Buy RSI,Buy Upper BB,Buy Bollinger %b,Buy Lower BB,...,Sell Awesome Oscillator,Sell Ultimate Oscillator,Sell TSI,Sell Acum/Dist,Profit,Profit %,Duration,Ticker Cum Profit,Ticker Cum Profit %,Profitable
0,MMM,2023-06-01,78.15,0.000000,0.000000,0.000000,30.768463,81.253930,0.035208,74.188070,...,4.128971,15.650355,22.458525,-2.170200e+09,13.69,17.52,55,13.69,17.52,Yes
1,MMM,2023-09-26,78.90,0.000000,0.000000,0.000000,11.651657,89.682306,-0.004406,75.420694,...,0.907853,9.368463,-5.023564,-3.638899e+09,-0.45,-0.57,41,13.24,16.94,No
2,MMM,2024-01-24,80.80,0.000000,0.000000,0.000000,10.734452,94.772918,-0.311989,80.577082,...,-0.136529,53.365039,-9.388064,-4.387782e+09,-2.22,-2.75,47,11.02,14.10,No
0,AOS,2023-06-01,64.17,24.140920,20.572748,7.125260,27.823372,70.267760,0.024639,63.512240,...,4.062324,27.267543,28.568387,-1.075693e+08,12.39,19.31,48,12.39,19.31,Yes
1,AOS,2023-08-21,68.87,24.140920,20.572748,7.125260,11.277111,74.791816,0.017214,68.072184,...,-0.297029,35.764834,-3.445978,-1.715373e+08,0.39,0.57,51,12.78,19.92,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,ZBRA,2024-02-01,239.55,52.622380,21.377842,5.093924,35.624809,263.373686,0.127750,239.297314,...,10.391324,58.610041,10.108570,5.112419e+06,42.86,17.89,15,18.59,7.43,Yes
0,ZBH,2023-05-26,127.83,26.588114,14.948156,2.137456,12.420403,143.256759,0.006662,127.345241,...,3.571059,67.443670,15.505956,6.405941e+06,14.09,11.02,20,14.09,11.02,Yes
1,ZBH,2023-08-01,131.08,26.588114,14.948156,2.137456,13.322170,145.763365,-0.412053,136.048635,...,1.228441,55.586276,-1.829845,-1.301001e+07,-20.32,-15.50,107,-6.23,-4.87,No
0,ZTS,2023-05-26,165.66,32.942800,25.774693,15.320125,9.121795,191.645888,-0.064956,166.037112,...,5.442971,68.846333,17.901474,1.631326e+07,17.54,10.59,59,17.54,10.59,Yes



---
# Further research
---

### Below we retrieve individual stock data based on it appearing in the buy/sell list above

In [15]:

CSX = Ticker('CSX', rec.start, rec.end)
CSX_df = CSX.df
CSX_df

CSX.profitability


{'Net Income': 3715000.0,
 'ROAA': 8.811669829222012e-05,
 'ROAA Score': 1,
 'Operating Cash Flow': 5549000000.0,
 'Operating Cash Flow Score': 1,
 'ROA': 8.811669829222012e-05,
 'ROA PY': 0.00010106376526812464,
 'ROA Score': 1,
 'Accruals': 0.13075983651808204,
 'Accruals Score': 1,
 'Profitability Score': 4}

In [17]:
CSX.piotroski


{'Profitability Score': 4,
 'Leverage Score': 2,
 'Operating Efficiency Score': 1,
 'F-Score': 7}

In [19]:
CSX.ratios

{'current_ratio': 1.0496277915632755,
 'price_earnings_ratio': 19.88108,
 'projected_price_earnings_ratio': 16.79452,
 'profit_sales_ratio': 4.912339,
 'profit_margin': 0.25346002,
 'profit_book_ratio': 5.9389634,
 'profit_dividend_ratio': 0.013099999,
 'profit_earnings_growth': 1.91,
 'profit_sales_growth': -0.013}